## Detecting Merged Cells And Headers on fictitious bank statement

We will be using the modules below:
* amazon-textract-caller (https://pypi.org/project/amazon-textract-caller/) to invoke Amazon Textract API on our behalf
* amazon-textract-response-parser (http://%28https//pypi.org/project/amazon-textract-response-parser/) to parse the response payload
* amazoon-textract-prettyprinter (https://pypi.org/project/amazon-textract-prettyprinter/) to "pretty-print" tables

In [ ]:
!pip install boto3
!pip install amazon-textract-caller
!pip install amazon-textract-prettyprinter
!pip install trp

In [ ]:
import boto3
import json
import pandas as pd
from textractcaller import call_textract, Textract_Features
from textractprettyprinter.t_pretty_print import Pretty_Print_Table_Format, Textract_Pretty_Print, get_string, get_tables_string
from trp import Document
from trp.trp2 import TDocument, TDocumentSchema
from trp.t_pipeline import order_blocks_by_geo
from IPython.display import display

Let's initialize the boto3 session and then invoke textract_caller to perform the document processing API call and collect the response back on our behalf.

In [ ]:
session = boto3.Session(profile_name='<your_profile_name>')
documentName = "s3://textract-table-merged-cells-data-sample/Textract-MergeCell-Statement.pdf"
textract_json = call_textract(input_document=documentName, features = [Textract_Features.TABLES])

Let's pretty-print the response payload. As you can see, by default the date is not populated across all rows.

In [ ]:
print(get_string(textract_json=textract_json, output_type=[Textract_Pretty_Print.TABLES]))

Now let's load the response into an ordered document and scan the statement's table.

In [ ]:
t_doc = TDocumentSchema().load(textract_json)
ordered_doc = order_blocks_by_geo(t_doc)
trp_doc = Document(TDocumentSchema().dump(ordered_doc))

table_index = 1
dataframes = []

def combine_headers(top_h, bottom_h):
    bottom_h[3] = top_h[2] + " " + bottom_h[3]
    bottom_h[4] = top_h[2] + " " + bottom_h[4]

for page in trp_doc.pages:
    for table in page.tables:
        table_data = []
        headers = table.get_header_field_names()
        if(len(headers)>0):                                      #Let's retain the only table with headers
            print("Statememt headers: "+ repr(headers))
            top_header= headers[0]
            bottom_header = headers[1]
            combine_headers(top_header, bottom_header)           #The statement has two headers. let's combine them
            for r, row in enumerate(table.rows_without_header):  #New Table attribute returning rows without headers
                table_data.append([])
                for c, cell in enumerate(row.cells):
                    table_data[r].append(cell.mergedText)        #New Cell attribute returning merged cells common values
            
            if len(table_data)>0:
                df = pd.DataFrame(table_data, columns=bottom_header)

df

Now we can even use multi level indexing and reproduce the table's initial structure.

In [ ]:
multi = df.set_index(['Date', 'Details'])
display(multi)